In [ ]:
import pandas as pd
import json

: 

In [ ]:
from nltk.corpus import wordnet

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
import nltk

In [ ]:
nltk.download('wordnet')

In [ ]:
nltk.download('stopwords')

In [ ]:
nltk.download('punkt_tab')


In [ ]:
with open('dataset/intents.json', 'r') as file:
    data = json.load(file)

# Convert 'intents' list to DataFrame
df = pd.DataFrame(data['intents'])

# Show DataFrame
df

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
import random

In [ ]:
def synonym_replacement(tokens, limit):
    augmented_sentences = []
    for i in range(len(tokens)):
        synonyms = []
        for syn in wordnet.synsets(tokens[i]):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if len(synonyms) > 0:
            num_augmentations = min(limit, len(synonyms))
            sampled_synonyms = random.sample(synonyms, num_augmentations)
            for synonym in sampled_synonyms:
                augmented_tokens = tokens[:i] + [synonym] + tokens[i+1:]
                augmented_sentences.append(' '.join(augmented_tokens))
    return augmented_sentences

In [ ]:
text_data=[]
labels=[]
stopwords= set(nltk.corpus.stopwords.words('english'))
lemmatizer=WordNetLemmatizer()

limit_per_tag=40

for intent in data['intents']:
    augmented_sentences_per_tag = 0
    for example in intent['patterns']:
        tokens = nltk.word_tokenize(example.lower())
        filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords and token.isalpha()]
        if filtered_tokens:
            text_data.append(' '.join(filtered_tokens))
            labels.append(intent['tag'])

            augmented_sentences = synonym_replacement(filtered_tokens, limit_per_tag - augmented_sentences_per_tag)
            for augmented_sentence in augmented_sentences:
                text_data.append(augmented_sentence)
                labels.append(intent['tag'])
                augmented_sentences_per_tag += 1
                if augmented_sentences_per_tag >= limit_per_tag:
                    break

In [ ]:
print(len(text_data))
print(len(labels))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_data)
y = labels

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model=LogisticRegression()

In [ ]:
l = model.fit(x_train, y_train)

In [ ]:
l

In [ ]:
L_pred = model.predict(x_test)

In [ ]:
accuracy_score(y_test, L_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model=RandomForestClassifier()

In [ ]:
R=model.fit(x_train,y_train)

In [ ]:
R

In [ ]:
R_pred = model.predict(x_test)

In [ ]:
accuracy_score(y_test, R_pred)

In [ ]:
from sklearn.svm import SVC

In [ ]:


model=SVC()

In [ ]:
S=model.fit(x_train,y_train)

In [ ]:
S

In [ ]:
s_pred=model.predict(x_test)

In [ ]:
accuracy_score(y_test,s_pred)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model=DecisionTreeClassifier()

In [ ]:
D=model.fit(x_train,y_train)

In [ ]:
D

In [ ]:
d_pred=model.predict(x_test)

In [ ]:
accuracy_score(y_test,d_pred)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:





accuracies = {
    'logistic': accuracy_score(y_test, L_pred),
    'svc': accuracy_score(y_test, s_pred),
    'DecisionTree': accuracy_score(y_test, d_pred),
    'RandomForest': accuracy_score(y_test, R_pred)
}
# Create a DataFrame from accuracies
accuracy_df = pd.DataFrame(list(accuracies.items()), columns=['Category', 'Value'])

# Create the bar plot
sns.barplot(x='Category', y='Value', data=accuracy_df)
plt.show()

In [ ]:
pip install pickle-mixin

In [ ]:
import pickle

In [ ]:
filename='model.pkl'

In [ ]:
with open (filename,'wb') as file:
  pickle.dump(R,file)

In [ ]:
filename='vectorizer.pkl'

In [ ]:
with open(filename,'wb') as file:
  pickle.dump(vectorizer,file)